In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('../Data/dataset.csv')

dataset.head(3)

,B0043GG,B0043HH,B0043II,Protein,Leukocyte,SG,Bilirubin,Acidity,Nitrite,Urobilinogen,...,Urin_color_Dk.Yellow,Urin_color_Lt. Yellow,Urin_color_Lt.Orange,Urin_color_Orange,Urin_color_Other,Urin_color_Red,Urin_color_Reddish,Urin_color_Straw,Urin_color_Yellow,Urin_color_yellow
0,9.3,251.5,4.3,1,119,1.020,0.1,5.0,103842,0.7,...,0,0,0,0,0,0,0,0,1,0
1,5.1,548.3,9.1,4,106,1.020,0.0,5.0,108098,0.7,...,0,0,0,0,0,0,0,0,1,0
2,12.2,1590.0,23.5,6,657,1.016,0.1,5.0,27515,0.7,...,0,0,0,0,0,0,0,0,1,0


In [3]:
dataset.corr()['Ketone_body'].abs().sort_values(ascending=False)

Ketone_body              1.000000
SG                       0.298067
Glucose                  0.286262
Bilirubin                0.168319
Acidity                  0.140239
Protein                  0.124388
Leukocyte                0.122045
Occult_blood             0.118204
B0043HH                  0.098697
Urin_color_Yellow        0.076316
Urin_color_Lt. Yellow    0.046562
Urin_color_Red           0.045988
B0043GG                  0.045945
Urin_color_yellow        0.044169
Urin_color_Dk.Yellow     0.038345
Urobilinogen             0.037152
B0043II                  0.036294
Urin_color_DK.Orange     0.035227
Urin_color_Orange        0.025040
Urin_color_Reddish       0.022312
Urin_color_Lt.Orange     0.014625
Nitrite                  0.012623
Urin_color_Other         0.008965
Urin_color_Amber         0.008579
Urin_color_Brown         0.006064
Urin_color_DK.Yellow     0.006058
Urin_color_Dk.Red        0.004395
Urin_color_Straw         0.002884
Urin_color_Dk. Yellow    0.001916
Urin_color_Dk.

In [5]:
# Selecing columns

dataset = dataset[['SG', 'Glucose', 'Bilirubin', 'Acidity',
                   'Protein', 'Leukocyte', 'Occult_blood', 'Ketone_body']]

dataset.head(3)

,SG,Glucose,Bilirubin,Acidity,Protein,Leukocyte,Occult_blood,Ketone_body
0,1.020,16,0.1,5.0,1,119,3,1
1,1.020,65,0.0,5.0,4,106,2,1
2,1.016,79,0.1,5.0,6,657,40,3


In [6]:
dataset.shape

(74337, 8)

In [7]:
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

X.shape, y.shape

((74337, 7), (74337,))

# Train, Test Split

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)

X_train.shape, X_test.shape

((59469, 7), (14868, 7))

In [13]:
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2022)

# X_train.shape, X_val.shape

((47575, 3), (11894, 3))

# Data Scaling

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
# X_val = ss.transform(X_val)
X_test = ss.transform(X_test)

In [12]:
X_train[:5, :]

array([[-0.59688789, -0.6161976 ,  0.19038558,  0.27419382, -0.4568329 ,
        -0.53870365, -0.68592602],
       [-0.34512929, -0.55352856, -0.54462091, -0.87977981, -0.44443037,
        -0.48245593, -0.55110695],
       [-1.60392227, -0.65403552,  0.55788882, -0.87977981, -0.48411847,
        -0.55628106,  1.72284137],
       [ 0.91366369, -0.55352856, -0.54462091, -0.87977981, -0.44443037,
        -0.48245593,  2.00596142],
       [-0.84864648, -0.60792056,  0.55788882, -0.49512194,  0.03926827,
        -0.40863081, -0.12867386]])

# Modeling

## 1. RandomForestRegressor

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold

In [14]:
params = {
    'n_estimators' : range(100, 200, 5),
    'verbose' : [True]
}

In [15]:
rfr = RandomForestRegressor(n_jobs=8, random_state=2022)

In [16]:
# 0값 확인
# 0값 존재 -> MAPE 사용 불가 (실제 y값으로 나누기 때문(y > 0 필수))

len(dataset.loc[dataset.Ketone_body == 0])

24956

In [17]:
kfold = KFold(5, shuffle=True, random_state=2022)

### Learning

In [18]:
grid = GridSearchCV(rfr, params, n_jobs=8, cv=kfold, refit=True)
grid.fit(X_train, y_train)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 195 out of 195 | elapsed:    3.5s finished


GridSearchCV(cv=KFold(n_splits=5, random_state=2022, shuffle=True),
             estimator=RandomForestRegressor(n_jobs=8, random_state=2022),
             n_jobs=8,
             param_grid={'n_estimators': range(100, 200, 5), 'verbose': [True]})

In [19]:
grid.best_params_

{'n_estimators': 195, 'verbose': True}

In [20]:
y_pred = grid.best_estimator_.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 195 out of 195 | elapsed:    0.0s finished


### Evaluating

In [21]:
from sklearn.metrics import mean_absolute_percentage_error, \
                            mean_absolute_error, mean_squared_error

In [22]:
mean_absolute_percentage_error(y_test, y_pred)

3.3122487288731544e+16

In [23]:
mean_absolute_error(y_test, y_pred)

16.15556977920403

In [24]:
np.sqrt(mean_squared_error(y_test, y_pred))

24.406912130850476

### Comparising y_pred, y_result

In [25]:
comparison = pd.DataFrame({'prediction' : y_pred, 'result' : y_test.values.ravel()})

In [26]:
comparison

,prediction,result
0,39.372012,0
1,0.000000,0
2,38.192399,2
3,37.859863,0
4,3.000000,3
...,...,...
14863,0.000000,0
14864,36.194741,75
14865,1.640459,0
14866,2.000000,2


### Saving Model

In [27]:
import pickle

In [28]:
saved_model = pickle.dump(grid.best_estimator_, open('../Model/{0}_RFT_n_{1}.pkl' \
    .format(dataset.columns[-1], grid.best_params_.get('n_estimators')), 'wb'),
                          protocol=pickle.HIGHEST_PROTOCOL)

### Loading Model

In [29]:
loaded_model = pickle.load(open('../Model/{0}_RFT_n_{1}.pkl' \
    .format(dataset.columns[-1], grid.best_params_.get('n_estimators')), 'rb'))

In [30]:
y_pred = loaded_model.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 195 out of 195 | elapsed:    0.0s finished


In [31]:
mean_absolute_error(y_test, y_pred)

16.15556977920403

In [32]:
np.sqrt(mean_squared_error(y_test, y_pred))

24.406912130850476